In [1]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install llama-index-llms-llama-cpp

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install --upgrade typing-extensions
!pip install --upgrade pip

    extract-msg (<=0.29.*)
                 ~~~~~~~^
    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [4]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/Users/sithukaungset/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sithukaungset/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#Llama CPP, in this notebook, we use the llama-2-chat-13b-ggml model, along with the proper prompt formatting.

!pip install llama-cpp-python

    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [9]:
from llama_index.llms.llama_cpp import LlamaCPP

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url = model_url,
    # optionally, we can set the path to a pre-downloaded model instead of model_url
    model_path = None,
    temperature = 0.1,
    max_new_tokens = 256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow to allow for some wiggle room
    context_window = 3900,
    # kwargs to pass to __call__
    generate_kwargs = {},
    # kwargs to pass to __init__
    # set to at least 1 to use GPU
    model_kwargs = {"n_gpu_layers": 1},
    verbose=True,
)


total size (MB): 7365.83


 50%|█████     | 3533/7024 [03:15<03:13, 18.03it/s]


Error downloading model: [Errno 28] No space left on device
Download incomplete. Removing partially downloaded file.


ValueError: Download incomplete.

In [ ]:
# init Postgres 
# Using an existing Postgres running at the localhost, create the database we will be using
!pip install psycopg2-binary pgvector asycpg "sqlalchemy[asyncio]" greenlet


In [ ]:
import psycopg2

dbname = "vector_db"
host = "localhost"
password = "password"
port = "5432"
user = "jerry"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname=dbname, 
    user=user, 
    password=password, 
    host=host, 
    port=port,
    user=user)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {dbname}")
    c.execute(f"CREATE DATABASE {dbname}")

In [ ]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port, 
    user=user,
    table_name="llama2_paper",
    embed_dim=384, # Openai embedding dimension
)

In [ ]:
# Build an Ingestion Pipeline from Scratch
# 1. Load Data
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

In [ ]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [ ]:
loader = PyMuPDFReader()
documents = loader.load(file_path=Path("data/llama2.pdf"))

In [ ]:
# 2. Use a Text Splitter to split the documents
from llama_index.core.node_parser import SentenceSplitter


In [ ]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator = "",
)


In [ ]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)

doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [ ]:
# 3. Manually Construct Nodes from Text Chunks
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    nodes = TextNode(
        text = text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc